In [1]:
#from keras import backend as K
import numpy as np
import pandas as pd
import os
from keras.preprocessing import image   
from keras.applications import xception
import matplotlib.image as mpimg
from PIL import ImageFile
from matplotlib.pyplot import imshow
from tqdm import tqdm
from mpl_toolkits.axes_grid1 import ImageGrid
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.utils import shuffle
from sklearn import preprocessing
import time
import keras
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Flatten, Dense
from keras.models import Sequential, Model
from keras.layers.normalization import BatchNormalization
from keras import regularizers, applications, optimizers, initializers
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.


In [2]:
!ls /Users/troyxiao/Desktop/ANLY590-DeepLearning/project/all

cleanedData             sample_submission.csv   train
keras-pretrained-models test                    train.zip
rootkey.csv             test.zip


In [3]:
CATEGORIES = ['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Common wheat', 'Fat Hen', 'Loose Silky-bent',
              'Maize', 'Scentless Mayweed', 'Shepherds Purse', 'Small-flowered Cranesbill', 'Sugar beet']
NUM_CATEGORIES = len(CATEGORIES)

In [6]:
data_dir = '/Users/troyxiao/Desktop/ANLY590-DeepLearning/project/all'
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')
#sample_submission = pd.read_csv(os.path.join(data_dir, 'sample_submission.csv'))

In [7]:
for category in CATEGORIES:
    print('{} {} images'.format(category, len(os.listdir(os.path.join(train_dir, category)))))

Black-grass 263 images
Charlock 391 images
Cleavers 287 images
Common Chickweed 611 images
Common wheat 221 images
Fat Hen 475 images
Loose Silky-bent 654 images
Maize 221 images
Scentless Mayweed 516 images
Shepherds Purse 231 images
Small-flowered Cranesbill 496 images
Sugar beet 385 images


In [8]:
train = []
for category_id, category in enumerate(CATEGORIES):
    for file in os.listdir(os.path.join(train_dir, category)):
        if file != '.DS_Store':
            train.append(['train/{}/{}'.format(category, file), category_id, category])
train = pd.DataFrame(train, columns=['file', 'category_id', 'category'])
train.shape

(4750, 3)

In [9]:
train.head(5)

,file,category_id,category
0,train/Black-grass/5a1295fb4.png,0,Black-grass
1,train/Black-grass/d8afd58f3.png,0,Black-grass
2,train/Black-grass/418808d19.png,0,Black-grass
3,train/Black-grass/eac39cfa8.png,0,Black-grass
4,train/Black-grass/39e9bf4c7.png,0,Black-grass


In [10]:
train = shuffle(train)
train.head(5)

,file,category_id,category
3651,train/Shepherds Purse/9db0aa841.png,9,Shepherds Purse
3554,train/Scentless Mayweed/0dd73ae6a.png,8,Scentless Mayweed
2040,train/Fat Hen/f0e75e3bf.png,5,Fat Hen
4122,train/Small-flowered Cranesbill/8d4967856.png,10,Small-flowered Cranesbill
1343,train/Common Chickweed/2e4aad6ec.png,3,Common Chickweed


In [11]:
# convert category_id to binary label
id = np.array(train['category_id'])
trainL = np.array([0,0,0,0,0,0,0,0,0,0,0,0])
for i in range(train.shape[0]):
    temp = np.array([0,0,0,0,0,0,0,0,0,0,0,0])
    temp[id[i]]=1
    trainL = np.vstack((trainL,temp))
trainL = np.delete(trainL,(0), axis=0)

In [12]:
for i in range(5):
    print(trainL[i])

[0 0 0 0 0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 0 1 0 0 0]
[0 0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1 0]
[0 0 0 1 0 0 0 0 0 0 0 0]


In [13]:
test = []
for file in os.listdir(test_dir):
    test.append(['test/{}'.format(file), file])
test = pd.DataFrame(test, columns=['filepath', 'file'])
test.shape

(794, 2)

In [14]:
test.head(2)

,filepath,file
0,test/1b490196c.png,1b490196c.png
1,test/85431c075.png,85431c075.png


In [15]:
# Helper method to convert images to training tensors 
def path_to_tensor(img_path, shape):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=shape)
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)/255
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

# Convert images to training tensors 
def paths_to_tensor(img_paths, shape):
    list_of_tensors = [path_to_tensor(img_path, shape) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [16]:
img_shape = (70,70)
#files_list = (data_dir + '/' + train['file']).tolist()
files_list=[]
for i in range(train.shape[0]):
    files_list.append(data_dir + '/' + train['file'][i])
    
test_files_list=[]
for i in range(test.shape[0]):
    test_files_list.append(data_dir + '/' + test['filepath'][i])

divide = 4000
train_tensors = paths_to_tensor(files_list[:divide], shape = img_shape)
valid_tensors = paths_to_tensor(files_list[divide:], shape = img_shape)
test_tensors = paths_to_tensor(test_files_list[:], shape = img_shape)

train_labels = trainL[:divide]
valid_labels = trainL[divide:]

100%|██████████| 794/794 [00:05<00:00, 155.89it/s]


In [17]:
base_model = applications.vgg16.VGG16(weights='imagenet',
                                               include_top=False,
                                               input_shape=train_tensors.shape[1:])

for layer in base_model.layers:
    layer.trainable = False

add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dropout(0.2))
#add_model.add(Dense(256, activation='relu'))
#add_model.add(Dropout(0.2))
add_model.add(Dense(64, activation='relu'))
add_model.add(Dropout(0.2))
add_model.add(Dense(12, activation='softmax'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))

model.summary()
add_model.summary()

58900480/58889256 [==============================] - 5s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 70, 70, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 70, 70, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 70, 70, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 35, 35, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 35, 35, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 35, 35, 128)       147584    
_________________________________________________________________
block2_pool

In [18]:
import keras.backend as K

model.compile(optimizer=optimizers.SGD(lr=1e-4, decay=1e-6, momentum=0.9, nesterov=True), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [20]:
from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping
import numpy as np

epochs = 5
batch_size = 16

earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=4, verbose=1, mode='auto')
log = CSVLogger('plant-seedling-log.csv')
checkpointer = ModelCheckpoint(filepath='plant-seedling-checkpoint.hdf5', 
                               verbose=1, save_best_only=True)

start = time.time()

def train_generator(x, y, batch_size):
        train_datagen = ImageDataGenerator(
                        shear_range=0.2,
                        zoom_range=0.2,
                        rotation_range=180,  # randomly rotate images in the range (degrees, 0 to 180)
                        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
                        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
                        horizontal_flip=True,  # randomly flip images
                        vertical_flip=True)  # randomly flip images
        generator = train_datagen.flow(x, y, batch_size=batch_size)
        while 1:
            x_batch, y_batch = generator.next()
            yield [x_batch, y_batch]

# Training with data augmentation. If shift_fraction=0., also no augmentation.
model.fit_generator(generator=train_generator(train_tensors, train_labels, batch_size),
                    steps_per_epoch=int(train_labels.shape[0] / batch_size),
                    validation_data=(valid_tensors, valid_labels),
                    epochs=epochs, callbacks=[checkpointer, log, earlystop], verbose=1)

# Show total training time
print("training time: %.2f minutes"%((time.time()-start)/60))

Epoch 1/5
250/250 [==============================] - 166s 662ms/step - loss: 2.4709 - acc: 0.1180 - val_loss: 2.4556 - val_acc: 0.1293

Epoch 00001: val_loss improved from inf to 2.45565, saving model to plant-seedling-checkpoint.hdf5
Epoch 2/5
250/250 [==============================] - 166s 665ms/step - loss: 2.4539 - acc: 0.1187 - val_loss: 2.4513 - val_acc: 0.1173

Epoch 00002: val_loss improved from 2.45565 to 2.45126, saving model to plant-seedling-checkpoint.hdf5
Epoch 3/5
250/250 [==============================] - 178s 712ms/step - loss: 2.4518 - acc: 0.1227 - val_loss: 2.4515 - val_acc: 0.1107

Epoch 00003: val_loss did not improve from 2.45126
Epoch 4/5
250/250 [==============================] - 171s 685ms/step - loss: 2.4431 - acc: 0.1293 - val_loss: 2.4520 - val_acc: 0.1107

Epoch 00004: val_loss did not improve from 2.45126
Epoch 5/5
250/250 [==============================] - 167s 668ms/step - loss: 2.4399 - acc: 0.1283 - val_loss: 2.4516 - val_acc: 0.1173

Epoch 00005: val

In [21]:
prediction = model.predict(test_tensors)
print(prediction)

[[0.05570696 0.09159406 0.05390646 ... 0.04794188 0.08568719 0.09007218]
 [0.05728008 0.09098065 0.0551459  ... 0.04546761 0.09137298 0.09057508]
 [0.06452724 0.09281871 0.05311167 ... 0.05057526 0.09028183 0.0766884 ]
 ...
 [0.0627821  0.09324825 0.05229995 ... 0.04123024 0.08517592 0.09381063]
 [0.05540825 0.08489912 0.05167867 ... 0.04585934 0.09018708 0.0844086 ]
 [0.06531483 0.08955759 0.05486565 ... 0.03931829 0.08831201 0.09971489]]


In [22]:
pred = np.argmax(prediction,axis=1)
predStr = []
for i in range(len(pred)):
    predStr.append(CATEGORIES[pred[i]])
preddf = pd.DataFrame({'species':predStr})

result=pd.concat([test[['file']], preddf], axis=1)
result.head(5)

,file,species
0,1b490196c.png,Common Chickweed
1,85431c075.png,Loose Silky-bent
2,506347cfe.png,Loose Silky-bent
3,7f46a71db.png,Loose Silky-bent
4,668c1007c.png,Loose Silky-bent


In [23]:
# The model almost always predict the largest two species 
result.head(10)

,file,species
0,1b490196c.png,Common Chickweed
1,85431c075.png,Loose Silky-bent
2,506347cfe.png,Loose Silky-bent
3,7f46a71db.png,Loose Silky-bent
4,668c1007c.png,Loose Silky-bent
5,71f5323c5.png,Common Chickweed
6,1f3f44563.png,Loose Silky-bent
7,beebe5f4e.png,Common Chickweed
8,780defa2e.png,Common Chickweed
9,df521c0c0.png,Loose Silky-bent
